# Bert Extractor quickstart
This notebook is meant as a quickstart of this package and how to set it.

In [1]:
from bert_extractor.extractors import ReviewsExtractor

Create a configuration dictionary this can be set with in the configs files.

In [2]:
configurations = {
    "configs": "tests"
}

### Create an object of the ReviewExtractor

In [ ]:
reviews = ReviewsExtractor(**configurations)

### Extract and preprocess data

In [ ]:
url = ""
tensor_extracted = reviews.extract_preprocess(url)

### Fit BERT model with extracted tensor

In [ ]:
tensor_extracted